In [2]:
import numpy as np
import cv2

from scipy import ndimage, misc
import os, glob
import sys
import skimage
from skimage import feature, data, io

from matplotlib.pyplot import imread, imsave
from PIL import Image
import random

In [17]:
def get_file_name_dir(rootDir, ext):
 
    # Return file names with certain extension

    return glob.glob1(rootDir, "*." + ext)


def clip(img, dtype, maxval):
    return np.clip(img, 0, maxval).astype(dtype)

def HorizontalFlip(img):
    prob=0.5
    if random.random() < prob:
        img = cv2.flip(img, 1)
    return img


def VerticalFlip(img):
    prob=0.5
    if random.random() < prob:
        img = cv2.flip(img, 0)
    return img

def RandomRotate(img):
    prob=0.5
    if random.random() < prob:
            factor = random.randint(0, 4)
            img = np.rot90(img, factor)
    return img

def RandomContrast(img):
    prob=0.8
    limit=.1
    if random.random() < prob:
        alpha = 1.0 + limit * random.uniform(-1, 1)

        gray = cv2.cvtColor(img[:, :, :3], cv2.COLOR_BGR2GRAY)
        gray = (3.0 * (1.0 - alpha) / gray.size) * np.sum(gray)
        maxval = np.max(img[..., :3])
        dtype = img.dtype
        img[:, :, :3] = clip(alpha * img[:, :, :3] + gray, dtype, maxval)
    return img

def RandomBrightness(img):
    prob=0.8
    limit=.1
    if random.random() < prob:
        alpha = 1.0 + limit * random.uniform(-1, 1)

        maxval = np.max(img[..., :3])
        dtype = img.dtype
        img[..., :3] = clip(alpha * img[..., :3], dtype, maxval)
    return img

def RandomCrop(img):
    prob=0.8
    limit=.1
    size= (25,25)
    h = size[0]
    w = size[1]
    if random.random() < prob:
        height, width, _ = img.shape

        h_start = np.random.randint(0, height - h)
        w_start = np.random.randint(0, width - w)

        img = img[h_start: h_start + h, w_start: w_start + w]

        assert img.shape[0] == h
        assert img.shape[1] == w
    return img

def CenterCrop(img):
    prob=0.8
    limit=.1
    height, width = (450,450)
    h, w, c = img.shape
    dy = (h - height) // 2
    dx = (w - width) // 2

    y1 = dy
    y2 = y1 + height
    x1 = dx
    x2 = x1 + width
    img = img[y1:y2, x1:x2]

    return img

def RandomHueSaturationValue(img):
    prob=0.8
    hue_shift_limit=(-10, 10)
    sat_shift_limit=(-25, 25)
    val_shift_limit=(-25, 25)
    if random.random() < prob:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(img)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        img = cv2.merge((h, s, v))
        img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return img
def RandomSaturation(img):
#     def __init__(self, limit=0.3, prob=0.5):
#         self.limit = limit
#         self.prob = prob

#     def __call__(self, img):
#         # dont work :(
#         if random.random() < self.prob:
    limit=0.3
    maxval = np.max(img[..., :3])
    dtype = img.dtype
    alpha = 2.0 + random.uniform(-limit, limit)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    img[..., :3] = alpha * img[..., :3] + (1.0 - alpha) * gray
    img[..., :3] = clip(img[..., :3], dtype, maxval)
    return img

def RandomFilter(img):
    """
    blur sharpen, etc
    """

    limit=.5
    alpha = limit * random.uniform(0, 1)
    kernel = np.ones((3, 3), np.float32) / 9 * 0.2

    colored = img[..., :3]
    colored = alpha * cv2.filter2D(colored, -1, kernel) + (1 - alpha) * colored
    maxval = np.max(img[..., :3])
    dtype = img.dtype
    img[..., :3] = clip(colored, dtype, maxval)

    return img
def ShiftScale(img):
    limit=4

    height, width, channel = img.shape
    assert (width == height)
    size0 = width
    size1 = width + 2 * limit
    size = round(random.uniform(size0, size1))

    dx = round(random.uniform(0, size1 - size))
    dy = round(random.uniform(0, size1 - size))

    y1 = dy
    y2 = y1 + size
    x1 = dx
    x2 = x1 + size

    img1 = cv2.copyMakeBorder(img, limit, limit, limit, limit, borderType=cv2.BORDER_REFLECT_101)
    img = (img1[y1:y2, x1:x2, :] if size == size0
    else cv2.resize(img1[y1:y2, x1:x2, :], (size0, size0), interpolation=cv2.INTER_LINEAR))

    return img

def shift(img):
    limit=4
    dx = round(random.uniform(-limit, limit))
    dy = round(random.uniform(-limit, limit))

    height, width, channel = img.shape
    y1 = limit + 1 + dy
    y2 = y1 + height
    x1 = limit + 1 + dx
    x2 = x1 + width

    img1 = cv2.copyMakeBorder(img, limit + 1, limit + 1, limit + 1, limit + 1,
                              borderType=cv2.BORDER_REFLECT_101)
    img = img1[y1:y2, x1:x2, :]


    return img

In [22]:
"""
Get name images to preprocessing
--------------------------------
"""
input_path = 'data/train/Nuclear_membrane/'
input_extension = 'png'

inputs_files = sorted(get_file_name_dir(input_path, input_extension))

print(('**')*30)
print('********************* Preprocessing *************************')
print(('**')*30)

"""
Iterate over the list of images
-------------------------------
"""
cont = 1

for file_name in inputs_files:
    print('Preprocessing: ' + file_name + ': ' + str(cont))
    cont += 1
    # Get the image
    img = input_path + file_name
    im_in = cv2.imread(img)

    # Crop Image
    image = RandomHueSaturationValue(im_in)
#     image = Image.fromarray(image)
     # Save Image
    cv2.imwrite('data/train/Nuclear_membrane/' + 'RHSV_' +file_name , image)

************************************************************
********************* Preprocessing *************************
************************************************************
Preprocessing: 58359_img.png: 1
Preprocessing: 58360_img.png: 2
Preprocessing: 58361_img.png: 3
Preprocessing: 58362_img.png: 4
Preprocessing: 58366_img.png: 5
Preprocessing: 58367_img.png: 6
Preprocessing: 58368_img.png: 7
Preprocessing: 58370_img.png: 8
Preprocessing: 58371_img.png: 9
Preprocessing: 58372_img.png: 10
Preprocessing: 58375_img.png: 11
Preprocessing: 58376_img.png: 12
Preprocessing: 58377_img.png: 13
Preprocessing: 58378_img.png: 14
Preprocessing: 58379_img.png: 15
Preprocessing: 58381_img.png: 16
Preprocessing: 58382_img.png: 17
Preprocessing: 58383_img.png: 18
Preprocessing: 58384_img.png: 19
Preprocessing: 58386_img.png: 20
Preprocessing: 58387_img.png: 21
Preprocessing: 58389_img.png: 22
Preprocessing: 58392_img.png: 23
Preprocessing: 58393_img.png: 24
Preprocessing: 58394_img.png: 25


Preprocessing: 58797_img.png: 339
Preprocessing: 58800_img.png: 340
Preprocessing: 58801_img.png: 341
Preprocessing: 58802_img.png: 342
Preprocessing: 58803_img.png: 343
Preprocessing: 58804_img.png: 344
Preprocessing: 58805_img.png: 345
Preprocessing: 58806_img.png: 346
Preprocessing: 58807_img.png: 347
Preprocessing: 58809_img.png: 348
Preprocessing: 58810_img.png: 349
Preprocessing: 58811_img.png: 350
Preprocessing: 58812_img.png: 351
Preprocessing: 58814_img.png: 352
Preprocessing: 58816_img.png: 353
Preprocessing: 58820_img.png: 354
Preprocessing: 58821_img.png: 355
Preprocessing: 58822_img.png: 356
Preprocessing: 58823_img.png: 357
Preprocessing: 58824_img.png: 358
Preprocessing: 58825_img.png: 359
Preprocessing: 58826_img.png: 360
Preprocessing: 58827_img.png: 361
Preprocessing: 58828_img.png: 362
Preprocessing: 58830_img.png: 363
Preprocessing: 58832_img.png: 364
Preprocessing: 58833_img.png: 365
Preprocessing: 58834_img.png: 366
Preprocessing: 58836_img.png: 367
Preprocessing:

Preprocessing: 59227_img.png: 668
Preprocessing: 59228_img.png: 669
Preprocessing: 59229_img.png: 670
Preprocessing: 59230_img.png: 671
Preprocessing: 59231_img.png: 672
Preprocessing: 59232_img.png: 673
Preprocessing: 59235_img.png: 674
Preprocessing: 59236_img.png: 675
Preprocessing: 59238_img.png: 676
Preprocessing: 59239_img.png: 677
Preprocessing: 59240_img.png: 678
Preprocessing: 59243_img.png: 679
Preprocessing: 59245_img.png: 680
Preprocessing: 59246_img.png: 681
Preprocessing: 59248_img.png: 682
Preprocessing: 59249_img.png: 683
Preprocessing: 59250_img.png: 684
Preprocessing: 59251_img.png: 685
Preprocessing: 59252_img.png: 686
Preprocessing: 59253_img.png: 687
Preprocessing: 59255_img.png: 688
Preprocessing: 59256_img.png: 689
Preprocessing: 59262_img.png: 690
Preprocessing: 59263_img.png: 691
Preprocessing: 59264_img.png: 692
Preprocessing: 59265_img.png: 693
Preprocessing: 59266_img.png: 694
Preprocessing: 59267_img.png: 695
Preprocessing: 59268_img.png: 696
Preprocessing:

Preprocessing: 59643_img.png: 997
Preprocessing: 59644_img.png: 998
Preprocessing: 59646_img.png: 999
Preprocessing: 59649_img.png: 1000
Preprocessing: 59651_img.png: 1001
Preprocessing: 59652_img.png: 1002
Preprocessing: 59653_img.png: 1003
Preprocessing: 59657_img.png: 1004
Preprocessing: 59658_img.png: 1005
Preprocessing: 59660_img.png: 1006
Preprocessing: 59662_img.png: 1007
Preprocessing: 59663_img.png: 1008
Preprocessing: 59664_img.png: 1009
Preprocessing: 59665_img.png: 1010
Preprocessing: 59666_img.png: 1011
Preprocessing: 59667_img.png: 1012
Preprocessing: 59668_img.png: 1013
Preprocessing: 59670_img.png: 1014
Preprocessing: 59671_img.png: 1015
Preprocessing: 59672_img.png: 1016
Preprocessing: 59673_img.png: 1017
Preprocessing: 59674_img.png: 1018
Preprocessing: 59675_img.png: 1019
Preprocessing: 59676_img.png: 1020
Preprocessing: 59679_img.png: 1021
Preprocessing: 59681_img.png: 1022
Preprocessing: 59682_img.png: 1023
Preprocessing: 59683_img.png: 1024
Preprocessing: 59684_im

Preprocessing: 60057_img.png: 1322
Preprocessing: 60058_img.png: 1323
Preprocessing: 60059_img.png: 1324
Preprocessing: 60060_img.png: 1325
Preprocessing: 60061_img.png: 1326
Preprocessing: 60062_img.png: 1327
Preprocessing: 60063_img.png: 1328
Preprocessing: 60065_img.png: 1329
Preprocessing: 60066_img.png: 1330
Preprocessing: 60067_img.png: 1331
Preprocessing: 60068_img.png: 1332
Preprocessing: 60071_img.png: 1333
Preprocessing: 60072_img.png: 1334
Preprocessing: 60074_img.png: 1335
Preprocessing: 60076_img.png: 1336
Preprocessing: 60077_img.png: 1337
Preprocessing: 60078_img.png: 1338
Preprocessing: 60081_img.png: 1339
Preprocessing: 60083_img.png: 1340
Preprocessing: 60085_img.png: 1341
Preprocessing: 60086_img.png: 1342
Preprocessing: 60087_img.png: 1343
Preprocessing: 60088_img.png: 1344
Preprocessing: 60089_img.png: 1345
Preprocessing: 60090_img.png: 1346
Preprocessing: 60091_img.png: 1347
Preprocessing: 60092_img.png: 1348
Preprocessing: 60094_img.png: 1349
Preprocessing: 60095

...

Preprocessing: RF_RB_63291_img.png: 23809
Preprocessing: RF_RB_63292_img.png: 23810
Preprocessing: RF_RB_63293_img.png: 23811
Preprocessing: RF_RB_63294_img.png: 23812
Preprocessing: RF_RB_63295_img.png: 23813
Preprocessing: RF_RB_63296_img.png: 23814
Preprocessing: RF_RB_63297_img.png: 23815
Preprocessing: RF_RB_63298_img.png: 23816
Preprocessing: RF_RB_63299_img.png: 23817
Preprocessing: RF_RB_63300_img.png: 23818
Preprocessing: RF_RB_63301_img.png: 23819
Preprocessing: RF_RB_63302_img.png: 23820
Preprocessing: RF_RB_63303_img.png: 23821
Preprocessing: RF_RB_63304_img.png: 23822
Preprocessing: RF_RB_63305_img.png: 23823
Preprocessing: RF_RB_63306_img.png: 23824
Preprocessing: RF_RB_63307_img.png: 23825
Preprocessing: RF_RB_63308_img.png: 23826
Preprocessing: RF_RB_63309_img.png: 23827
Preprocessing: RF_RB_63311_img.png: 23828
Preprocessing: RF_RB_63312_img.png: 23829
Preprocessing: RF_RB_63313_img.png: 23830
Preprocessing: RF_RB_63314_img.png: 23831
Preprocessing: RF_RB_63315_img.png

Preprocessing: RF_RC_58576_img.png: 24100
Preprocessing: RF_RC_58577_img.png: 24101
Preprocessing: RF_RC_58578_img.png: 24102
Preprocessing: RF_RC_58579_img.png: 24103
Preprocessing: RF_RC_58580_img.png: 24104
Preprocessing: RF_RC_58582_img.png: 24105
Preprocessing: RF_RC_58583_img.png: 24106
Preprocessing: RF_RC_58584_img.png: 24107
Preprocessing: RF_RC_58585_img.png: 24108
Preprocessing: RF_RC_58586_img.png: 24109
Preprocessing: RF_RC_58587_img.png: 24110
Preprocessing: RF_RC_58588_img.png: 24111
Preprocessing: RF_RC_58589_img.png: 24112
Preprocessing: RF_RC_58590_img.png: 24113
Preprocessing: RF_RC_58591_img.png: 24114
Preprocessing: RF_RC_58592_img.png: 24115
Preprocessing: RF_RC_58594_img.png: 24116
Preprocessing: RF_RC_58595_img.png: 24117
Preprocessing: RF_RC_58597_img.png: 24118
Preprocessing: RF_RC_58599_img.png: 24119
Preprocessing: RF_RC_58600_img.png: 24120
Preprocessing: RF_RC_58601_img.png: 24121
Preprocessing: RF_RC_58602_img.png: 24122
Preprocessing: RF_RC_58604_img.png

Preprocessing: RF_RC_58930_img.png: 24381
Preprocessing: RF_RC_58931_img.png: 24382
Preprocessing: RF_RC_58932_img.png: 24383
Preprocessing: RF_RC_58933_img.png: 24384
Preprocessing: RF_RC_58934_img.png: 24385
Preprocessing: RF_RC_58936_img.png: 24386
Preprocessing: RF_RC_58937_img.png: 24387
Preprocessing: RF_RC_58938_img.png: 24388
Preprocessing: RF_RC_58939_img.png: 24389
Preprocessing: RF_RC_58940_img.png: 24390
Preprocessing: RF_RC_58941_img.png: 24391
Preprocessing: RF_RC_58942_img.png: 24392
Preprocessing: RF_RC_58943_img.png: 24393
Preprocessing: RF_RC_58944_img.png: 24394
Preprocessing: RF_RC_58950_img.png: 24395
Preprocessing: RF_RC_58952_img.png: 24396
Preprocessing: RF_RC_58953_img.png: 24397
Preprocessing: RF_RC_58954_img.png: 24398
Preprocessing: RF_RC_58955_img.png: 24399
Preprocessing: RF_RC_58960_img.png: 24400
Preprocessing: RF_RC_58964_img.png: 24401
Preprocessing: RF_RC_58966_img.png: 24402
Preprocessing: RF_RC_58967_img.png: 24403
Preprocessing: RF_RC_58968_img.png

Preprocessing: RF_RC_59315_img.png: 24667
Preprocessing: RF_RC_59318_img.png: 24668
Preprocessing: RF_RC_59320_img.png: 24669
Preprocessing: RF_RC_59321_img.png: 24670
Preprocessing: RF_RC_59322_img.png: 24671
Preprocessing: RF_RC_59323_img.png: 24672
Preprocessing: RF_RC_59324_img.png: 24673
Preprocessing: RF_RC_59325_img.png: 24674
Preprocessing: RF_RC_59326_img.png: 24675
Preprocessing: RF_RC_59327_img.png: 24676
Preprocessing: RF_RC_59328_img.png: 24677
Preprocessing: RF_RC_59329_img.png: 24678
Preprocessing: RF_RC_59330_img.png: 24679
Preprocessing: RF_RC_59331_img.png: 24680
Preprocessing: RF_RC_59332_img.png: 24681
Preprocessing: RF_RC_59334_img.png: 24682
Preprocessing: RF_RC_59335_img.png: 24683
Preprocessing: RF_RC_59337_img.png: 24684
Preprocessing: RF_RC_59338_img.png: 24685
Preprocessing: RF_RC_59339_img.png: 24686
Preprocessing: RF_RC_59340_img.png: 24687
Preprocessing: RF_RC_59343_img.png: 24688
Preprocessing: RF_RC_59344_img.png: 24689
Preprocessing: RF_RC_59345_img.png

KeyboardInterrupt: 

In [21]:
len(os.listdir("data/train/Nuclear_membrane/"))

63824